In [1]:
import transformers
transformers.__version__

c:\Users\user\anaconda3\envs\a8\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


'4.36.2'

In [2]:
import trl
trl.__version__

c:\Users\user\anaconda3\envs\a8\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\user\anaconda3\envs\a8\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\user\anaconda3\envs\a8\lib\site-packages\trl\trainer\ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


'0.7.4'

In [3]:
import os
import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
!pip install fsspec==2023.9.2

In [6]:
# Step 1: Load the dataset
from datasets import load_dataset
train_ds = load_dataset('json', data_files='alpaca_data.json', split='train')
train_ds

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 52002
})

In [7]:
train_ds[1000]

{'input': 'Canada',
 'output': "The three historical events related to Canada are the Dominion of Canada's establishment in 1867, the adoption of the Canadian Flag in 1965, and the signing of the Canada-U.S. Free Trade Agreement in 1988.",
 'instruction': 'List 3 historical events related to the following country'}

In [8]:
eval_ds = load_dataset("tatsu-lab/alpaca_eval", split='eval', trust_remote_code=True)
eval_ds = eval_ds.remove_columns(["generator", "dataset"])
eval_ds

Dataset({
    features: ['instruction', 'output'],
    num_rows: 805
})

In [9]:
eval_ds[10]

{'instruction': 'do you think retinoid is effective on removing the acne? because I have a lot of it',
 'output': 'Yes, retinoids are effective in treating acne. They work by increasing cell turnover, which helps to reduce the appearance of existing acne and prevent new outbreaks. Retinoids also help to unclog pores, which in turn reduces the amount of bacteria that can cause infections. In general, retinoids help to reduce inflammation and oil production, making them a great option for those with acne.'}

In [9]:
# Step 2: Load the model & Tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name_or_path = "distilgpt2"
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path, device_map = device)
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

c:\Users\user\anaconda3\envs\a8\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--distilgpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\user\anaconda3\envs\a8\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._regis

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Standard-Alpaca

In [10]:
def formatting_prompts_func(examples):
    output_texts = [
        (f"""
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{examples["instruction"][i]}

### Input:
{examples["input"][i] if 'input' in examples.keys() and len(examples["input"][i]) > 1 else ""}

### Response:
{examples["output"][i]}
""").strip() for i in range(len(examples['instruction']))
    ]

    return output_texts

In [11]:
#check instruction-prompt
formatting_prompts_func(train_ds[:2])

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Input:\n\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat are the three primary colors?\n\n### Input:\n\n\n### Response:\nThe three primary colors are red, blue, and yellow.']

In [12]:
# use the DataCollatorForCompletionOnlyLM to train your model on the generated prompts only
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

response_template = "### Response:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
collator

DataCollatorForCompletionOnlyLM(tokenizer=GPT2TokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}, mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

**Training**

In [13]:
# Make sure to pass a correct value for max_seq_length as the default value will be set to min(tokenizer.model_max_length, 1024).
max_seq_length = min(tokenizer.model_max_length, 128)
max_seq_length

128

In [14]:
from transformers import TrainingArguments

save_path = './model/result'

training_args = TrainingArguments(
    output_dir = './model', 
    save_strategy = 'epoch',
    evaluation_strategy = 'epoch',
    gradient_checkpointing = True,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    num_train_epochs = 3, 
)

# Step 3: Define the Trainer
trainer = SFTTrainer(
    model,
    args = training_args,
    train_dataset = train_ds.select(range(100)),
    eval_dataset = eval_ds,
    formatting_func = formatting_prompts_func,
    data_collator = collator,
    max_seq_length = max_seq_length,
)

trainer.train()
trainer.save_model(save_path)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/805 [00:00<?, ? examples/s]

c:\Users\user\anaconda3\envs\a8\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/150 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
c:\Users\user\anaconda3\envs\a8\lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


...

{'eval_loss': nan, 'eval_runtime': 70.1909, 'eval_samples_per_second': 11.469, 'eval_steps_per_second': 5.741, 'epoch': 2.0}


c:\Users\user\anaconda3\envs\a8\lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


  0%|          | 0/403 [00:00<?, ?it/s]

c:\Users\user\anaconda3\envs\a8\lib\site-packages\trl\trainer\utils.py:120: UserWarning: Could not find response key `### Response:` in the following instance: Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Why is it that only proteins, carbohydrates and fats are deemed to have caloric value?<br>Just to clarify - consider the hypothetical situation below:<br>100g of 'carbohydrates' = 400kCal<br>100g of 'protein' = 400kCal<br>100g of 'fat' = 900kCal<br>100g of 'vitamins, minerals and/or other electrolytes' =?

### This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `max_seq_length`.
  warnings.warn(
c:\Users\user\anaconda3\envs\a8\lib\site-packages\trl\trainer\utils.py:120: UserWarning: Could not find response key `### Response:` in the following instance: Below is an instruction that describes a task, paired

{'eval_loss': nan, 'eval_runtime': 68.6307, 'eval_samples_per_second': 11.729, 'eval_steps_per_second': 5.872, 'epoch': 3.0}
{'train_runtime': 350.3214, 'train_samples_per_second': 0.856, 'train_steps_per_second': 0.428, 'train_loss': 2.288429972330729, 'epoch': 3.0}


**Inference**

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

save_path = './model/result'

model = AutoModelForCausalLM.from_pretrained(
    save_path,
    device_map = 'auto')

tokenizer = AutoTokenizer.from_pretrained(
    save_path)

text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map = 'auto',
    pad_token_id = tokenizer.eos_token_id,
    max_new_tokens = 100,
    temperature = 1.0
)


c:\Users\user\anaconda3\envs\a8\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\user\anaconda3\envs\a8\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [16]:
# def instruction_prompt(sample):
	
# 	if 'input' in sample.keys():
# 		return f"""
# 				Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# 				### Instruction:
# 				{sample['instruction']}

# 				### Input:
# 				{sample['input']}

# 				### Response:
# 				""".strip()
					
# 	else:
# 		return f"""
# 				Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# 				### Instruction:
# 				{sample['instruction']}

# 				### Response:
# 				""".strip()

In [16]:
def instruction_prompt(sample):
	
	if 'input' in sample.keys():
		return f"""
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{sample['instruction']}

### Input:
{sample['input']}

### Response:
""".strip()
			
	else:
		return f"""
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{sample['instruction']}

### Response:
""".strip()

In [17]:
import warnings
warnings.filterwarnings("ignore")

def comparision(pipeline, sample):
    print(f"Instruction:\n{sample['instruction']}\n")

    if 'input' in sample.keys():
        if len(sample['input']) > 1:
            print(f"Input:\n{sample['input']}\n")

    print(f"Gold Response:\n{sample['output']}\n")

    output = pipeline(instruction_prompt(sample))
    response = output[0]['generated_text'].split("### Response:\n")[-1]

    print(f"Generated Response:\n{response}\n")

In [22]:
comparision(text_generator, eval_ds[1])

Instruction:
How did US states get their names?

Gold Response:
US states get their names from a variety of sources, including Native American tribes, Spanish explorers, British colonists, and even presidents. For example, the state of Alabama was named after the Native American tribe that lived in the area, while the state of Florida gets its name from the Spanish explorer, Ponce de Leon, who explored the area in the 1500s. Other states are named after English kings (like Virginia, named after England's "Virgin Queen," Queen Elizabeth I) or presidents (like Washington, named after George Washington).

Generated Response:
The answer is simple: US states received their names from the following sources: the US states who submitted their names. The US governments who used the same names were the USA, Canada or Colombia.
The US states who submitted their names were in the states with the most number of names. Some of the countries were South America, Australia, and New Zealand, but was the